In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import time
from datetime import datetime
import tensorflow_addons as tfa
import keras_tuner as kt

2022-03-10 21:39:30.854964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-10 21:39:30.854985: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def exports():   
    # Set CUDA and CUPTI paths  
    os.environ['CUDA_HOME'] = '/usr/local/cuda'
    os.environ['PATH']= '/usr/local/cuda/bin:$PATH'  
    os.environ['CPATH'] = '/usr/local/cuda/include:$CPATH'  
    os.environ['LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LIBRARY_PATH'  
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/extras/CUPTI/lib64:$LD_LIBRARY_PATH'  
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LD_LIBRARY_PATH'
    os.environ['TF_GPU_THREAD_MODE']='gpu_private'

In [3]:
exports

<function __main__.exports()>

# Load data

In [4]:
cf15= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/cf15.csv', low_memory=False, header=None)
gf25= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf25.csv', low_memory=False, header=None)
gf34= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf34.csv', low_memory=False, header=None)
gm01= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01.csv', low_memory=False, header=None)
hc10= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/hc10.csv', low_memory=False, header=None)
nc46= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc46.csv', low_memory=False, header=None)
nc74= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc74.csv', low_memory=False, header=None)
nf11= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nf11.csv', low_memory=False, header=None)
no22= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/no22.csv', low_memory=False, header=None)
wc16= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc16.csv', low_memory=False, header=None)
wc41= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc41.csv', low_memory=False, header=None)
wo27= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo27.csv', low_memory=False, header=None)
wo34= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo34.csv', low_memory=False, header=None)
wo36= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36.csv', low_memory=False, header=None)
wo37= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo37.csv', low_memory=False, header=None)
wo40= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo40.csv', low_memory=False, header=None)
wo45= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo45.csv', low_memory=False, header=None)

In [5]:
val_1 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01_EarlyMarch.csv', low_memory=False, header=None)
val_2 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36_EarlyMarch.csv', low_memory=False, header=None)
val_3 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/go22.csv', low_memory=False, header=None)

In [6]:
frames =[cf15, gf25,gm01,nc46, nc74,nf11, wc16, wc41,wo27,wo34, wo36, wo37, wo40, wo45]
frames_2=[val_1, val_2, val_3]
data = pd.concat(frames)
val = pd.concat(frames_2)

# Define data cleaning functions

In [7]:
def snip(data):
    #Snip unwanted characters 

    itime= data.iloc[:,0]
    itime=itime.map(lambda x: str(x[6:]))
    itime=itime.map(lambda x: float(x))

    date=itime
    minute=date.map(lambda x: time.gmtime(x+7200)[4])
    hour=date.map(lambda x: time.gmtime(x+7200)[3])
    day=date.map(lambda x: time.gmtime(x+7200)[2])
    month=date.map(lambda x: time.gmtime(x+7200)[1])

    interface= data.iloc[:,-14]
    interface=interface.map(lambda x: str(x))
    interface=interface.map(lambda x: str(x[15:]))
    interface=interface.map(lambda x: str(x[:-1]))

    dev_id=data.iloc[:,3]
    dev_id=dev_id.map(lambda x: str(x))    
    dev_id=dev_id.map(lambda x: str(x[7:]))
    dev_id=dev_id.map(lambda x: str(x[:-1]))

    jitter=data.iloc[:, -13]
    jitter=jitter.map(lambda x: str(x))
    jitter=jitter.map(lambda x: str(x[8:]))
    jitter=jitter.map(lambda x: str(x[:-1]))
    jitter=jitter.map(lambda x: float(x))

    latency= data.iloc[:,-12]
    latency=latency.map(lambda x: str(x))
    latency=latency.map(lambda x: str(x[9:]))
    latency=latency.map(lambda x: str(x[:-1]))
    latency=latency.map(lambda x: float(x))

    inband= data.iloc[:,-15]
    inband=inband.map(lambda x: str(x))
    inband=inband.map(lambda x: str(x[22:]))
    inband=inband.map(lambda x: str(x[:-5]))
    inband=inband.map(lambda x: float(x))

    outband= data.iloc[:, -5]
    outband=outband.map(lambda x: str(x))
    outband=outband.map(lambda x: x.split('"')[1])
    outband=outband.map(lambda x: str(x[:-4]))
    outband=outband.map(lambda x: float(x))

    packet=data.iloc[:,-4]
    packet=packet.map(lambda x: str(x))
    packet=packet.map(lambda x: str(x[12:]))
    packet=packet.map(lambda x: str(x[:-2]))
    packet=packet.map(lambda x: float(x))

    status=data.iloc[:,-2]
    status=status.map(lambda x: str(x))
    status=status.map(lambda x: str(x[8:]))
    status=status.map(lambda x: str(x[:-1]))

    status[:] = np.where(status=="up", 1,0)
    status=status.map(lambda x: float(x))
    status=status.rolling(20).mean()
    status=status.map(np.floor)

    train_init=pd.concat([itime, minute, hour, day, month, interface, dev_id, inband, outband, latency, packet, jitter, status], axis=1)
    train_init.columns=['Time','Minute', 'Hour','Day','Month', 'Interface', 'Dev_id', 'Inband', 'Outband', 'Latency', 'Packet', 'Jitter', 'Status']

    #Dedupp based on interfaces
    train=train_init[train_init.Interface != 'NI2A-WWW1']
    train=train[train.Interface != 'NI2B-WWW1']
    train=train[train.Interface != 'NI2A-WWW2']
    train=train[train.Interface != 'NI2B-WWW2']
    train=train[train.Interface != 'NI2A-WWW3']    
    train=train[train.Interface != 'NI2B-WWW3']
    train=train[train.Interface != 'NI2A-MPLS']
    train['Interface']= train['Interface'].astype('category')
    train['Interface']=train['Interface'].cat.codes

    #Remove unwanted columns
    train_label=train.filter(['Status'])
    train=train.drop(['Status','Inband', 'Outband'], axis=1)
    train= train.sort_values(by=['Dev_id', 'Time'])
    train=train.reset_index(drop=True)
    train=train.drop(['Dev_id'], axis=1)

    return train, train_label

In [8]:
train,train_label=snip(data)
train=train.iloc[:-700, :]
train=train.to_numpy()
train_label=train_label.iloc[700:]

In [9]:
test,test_label=snip(val)
test=test.iloc[:-700, :]
test=test.to_numpy()
test_label=test_label.iloc[700:]

In [10]:
#train, train_label = preproccess(data)
mean=train.mean(axis=0)
train-=mean
std = train.std(axis=0)
train/=std
#test, test_label = preproccess(val)
test-=mean
test/=std

In [11]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

2022-03-10 21:40:16.881282: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-10 21:40:16.881309: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-10 21:40:16.883904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-10 21:40:16.884252: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


Build and train model

def get_model(hp):
    model=tf.keras.Sequential()
    hp_units= hp.Int('units', min_value=32, max_value=512, step=8)
    model.add(tf.keras.layers.Dense(units=hp_units, activation= 'relu'))
    model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
    if hp.Boolean("dropout"):
        model.add(tf.keras.layers.Dropout(rate=0.25))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='BinaryCrossentropy',
                metrics=tfa.metrics.F1Score(num_classes=1, average='micro'))
    
    return model


In [12]:
def get_model(hp):
    model=tf.keras.Sequential()
    #Tune number of layers
    for i in range(hp.Int("num_layers", 1, 4)):
        model.add(
            tf.keras.layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=0, max_value=320, step=8),
                activation=hp.Choice("activation", ["relu", "tanh"])
            )
        )
        if hp.Boolean("dropout"):
            model.add(tf.keras.layers.Dropout(rate=0.25))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='BinaryCrossentropy',
                metrics=tfa.metrics.F1Score(num_classes=1, average='micro'))
    
    return model


In [16]:
tuner = kt.Hyperband(get_model,
                     objective=kt.Objective("val_f1_score", direction="max"),
                     max_epochs=10,
                     factor=2,
                     directory='/home/jordan/Documents/DataSets/Data-Science/test3',
                     project_name='Tuning_test')

INFO:tensorflow:Reloading Oracle from existing project /home/jordan/Documents/DataSets/Data-Science/test3/Tuning_test/oracle.json


In [17]:
callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [18]:
tuner.search(train, train_label, epochs=50, validation_split=0.2,
callbacks= [callback])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 31 Complete [00h 03m 46s]
val_f1_score: 0.8205395936965942

Best val_f1_score So Far: 0.8472816348075867
Total elapsed time: 00h 57m 08s
INFO:tensorflow:Oracle triggered exit


In [20]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train, train_label, epochs=50, validation_split=0.2)

Epoch 1/50
17259/17259 [==============================] - 82s 5ms/step - loss: 4.5816 - f1_score: 0.7045 - val_loss: 4.3935 - val_f1_score: 0.8140
Epoch 2/50
17259/17259 [==============================] - 77s 4ms/step - loss: 4.6719 - f1_score: 0.7609 - val_loss: 3.8243 - val_f1_score: 0.7401
Epoch 3/50
17259/17259 [==============================] - 77s 4ms/step - loss: 5.2416 - f1_score: 0.6725 - val_loss: 2.4878 - val_f1_score: 0.7453
Epoch 4/50
17259/17259 [==============================] - 75s 4ms/step - loss: 5.0944 - f1_score: 0.6593 - val_loss: 2.0694 - val_f1_score: 0.8583
Epoch 5/50
17259/17259 [==============================] - 87s 5ms/step - loss: 5.3529 - f1_score: 0.6729 - val_loss: 1.7229 - val_f1_score: 0.8473
Epoch 6/50
17259/17259 [==============================] - 82s 5ms/step - loss: 4.4794 - f1_score: 0.7350 - val_loss: 3.9206 - val_f1_score: 0.7522
Epoch 7/50
17259/17259 [==============================] - 81s 5ms/step - loss: 3.9855 - f1_score: 0.7714 - val_loss: 3

KeyboardInterrupt: 

In [ ]:
val_acc_per_epoch = history.history["val_f1_score"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 136)               1360      
                                                                 
 dense_7 (Dense)             (None, 296)               40552     
                                                                 
 dense_8 (Dense)             (None, 224)               66528     
                                                                 
 dense_9 (Dense)             (None, 16)                3600      
                                                                 
Total params: 112,040
Trainable params: 112,040
Non-trainable params: 0
_________________________________________________________________


In [26]:
#hypermodel = tuner.hypermodel.build(best_hps)

#Retrain the model
model.fit(train, train, epochs=4, validation_split=0.2)

Epoch 1/4


ValueError: in user code:

    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, 

In [25]:
test_loss, test_acc =model.evaluate(test, test_label, verbose=2)
print('\nTest Accuracy:', test_acc)

2910/2910 - 3s - loss: 0.8560 - f1_score: 0.9709 - 3s/epoch - 1ms/step

Test Accuracy: 0.9709052443504333


In [21]:
model.save('tuned_model2.h5')